# Notebook to develop HMM model to investigate optimal angle selection

## Defining the targets we will be using in our multiaspect classification model

In [47]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.gridspec as gridspec

from scipy.special import spherical_jn, spherical_yn

import ipywidgets as widgets
from IPython.display import display

from sklearn.cluster import KMeans
import seaborn as sns
import pandas as pd

In [48]:
prior_target = 1/2
likelihood_nontarget = 1/2

In [49]:
B = np.array([[2/3, 2/3, 1, 1],
              [1/3, 1/3, 0, 0]])

In [50]:
likelihood_t1_o0 = (1/4 * B[0]).sum()
likelihood_t1_o0, 5/6

(np.float64(0.8333333333333333), 0.8333333333333334)

In [51]:
likelihood_t1_o1 = (1/4 * B[1]).sum()
likelihood_t1_o1, 1/6

(np.float64(0.16666666666666666), 0.16666666666666666)

In [52]:
post_t1 = (prior_target*likelihood_t1_o0) / ((prior_target*likelihood_t1_o0) + ((1-prior_target)*likelihood_nontarget))
post_t1

np.float64(0.625)

In [53]:
-((post_t1*np.log2(post_t1)) + ((1-post_t1)*np.log2(1-post_t1)))

np.float64(0.954434002924965)

In [54]:
s0_center = np.pi/4
state_steps = np.pi/2
state_centers = np.arange(s0_center, 2*np.pi, state_steps)
state_centers

array([0.78539816, 2.35619449, 3.92699082, 5.49778714])

In [55]:
angular_dist_between_states = (state_centers[np.newaxis, :] - state_centers[:, np.newaxis])
angular_dist_between_states

array([[ 0.        ,  1.57079633,  3.14159265,  4.71238898],
       [-1.57079633,  0.        ,  1.57079633,  3.14159265],
       [-3.14159265, -1.57079633,  0.        ,  1.57079633],
       [-4.71238898, -3.14159265, -1.57079633,  0.        ]])

In [56]:
candidate_step = -np.pi/2

angular_dist_between_states_negative_extent = angular_dist_between_states % (-2*np.pi)
angular_dist_after_action = (angular_dist_between_states_negative_extent - candidate_step)
angular_dist_after_action_negative_extent = angular_dist_after_action % (-2*np.pi)
sigma = state_steps/2
magnitude = 1 / np.sqrt((2*np.pi) * (sigma**2))
state_transition = magnitude * np.exp(-0.5*((angular_dist_after_action_negative_extent/sigma)**2))
state_transition_normed = state_transition / state_transition.sum(axis=1)
state_transition_normed.round(3)

array([[0.   , 0.   , 0.119, 0.881],
       [0.881, 0.   , 0.   , 0.119],
       [0.119, 0.881, 0.   , 0.   ],
       [0.   , 0.119, 0.881, 0.   ]])

In [57]:
alpha_t1 = np.array([1/6, 1/6, 1/4, 1/4])
alpha_t1

array([0.16666667, 0.16666667, 0.25      , 0.25      ])

In [58]:
alpha_t1 @ state_transition_normed

array([0.17662192, 0.24997538, 0.24004474, 0.16669128])

In [59]:
np.array([((0.88/6) + (0.12/4)), ((0.88/4) + (0.12/4)), ((0.88/4) + (0.12/6)), ((0.88/6) + (0.12/6))])

array([0.17666667, 0.25      , 0.24      , 0.16666667])

In [60]:
alpha_t2_o0 = (alpha_t1 @ state_transition_normed) * B[0]
alpha_t2_o0

array([0.11774795, 0.16665026, 0.24004474, 0.16669128])

In [61]:
alpha_t2_o1 = (alpha_t1 @ state_transition_normed) * B[1]
alpha_t2_o1

array([0.05887397, 0.08332513, 0.        , 0.        ])

In [62]:
likelihood_t2_o0 = (alpha_t2_o0).sum()
likelihood_t2_o0_given_o0t1 = likelihood_t2_o0 / likelihood_t1_o0
likelihood_t2_o0, likelihood_t2_o0_given_o0t1

(np.float64(0.6911342306668515), np.float64(0.8293610768002219))

In [63]:
likelihood_t2_o1 = (alpha_t2_o1).sum()
likelihood_t2_o1_given_o0t1 = likelihood_t2_o1 / likelihood_t1_o1
likelihood_t2_o1, likelihood_t2_o1_given_o0t1

(np.float64(0.1421991026664816), np.float64(0.8531946159988897))

In [64]:
post_t2_o0 = (likelihood_t2_o0_given_o0t1 * post_t1) / ((likelihood_t2_o0_given_o0t1 * post_t1) + (0.5 * (1-post_t1)))
post_t2_o0

np.float64(0.7343630782371399)

In [65]:
post_t2_o1 = (likelihood_t2_o1_given_o0t1 * post_t1) / ((likelihood_t2_o1_given_o0t1 * post_t1) + (0.5 * (1-post_t1)))
post_t2_o1

np.float64(0.7398531038578063)

In [66]:
entropy_t2_o0 = -((post_t2_o0*np.log2(post_t2_o0)) + ((1-post_t2_o0)*np.log2(1-post_t2_o0)))
entropy_t2_o0

np.float64(0.8351339850449211)

In [67]:
entropy_t2_o1 = -((post_t2_o1*np.log2(post_t2_o1)) + ((1-post_t2_o1)*np.log2(1-post_t2_o1)))
entropy_t2_o1

np.float64(0.826967959883645)

In [68]:
p_t2_o0 = ((post_t1*likelihood_t2_o0)/alpha_t1.sum()) + (((1-post_t1)*(1/4))/(1/2))
p_t2_o0

np.float64(0.7058506730001387)

In [69]:
p_t2_o1 = ((post_t1*likelihood_t2_o1)/alpha_t1.sum()) + (((1-post_t1)*(1/4))/(1/2))
p_t2_o1

np.float64(0.29414932699986124)

In [70]:
(p_t2_o0*entropy_t2_o0) + (p_t2_o1*entropy_t2_o1)

np.float64(0.8327319542394678)

In [71]:
candidate_step = np.pi/2

angular_dist_between_states_positive_extent = angular_dist_between_states % (2*np.pi)
angular_dist_after_action = (angular_dist_between_states_positive_extent - candidate_step)
angular_dist_after_action_positive_extent = angular_dist_after_action % (2*np.pi)
sigma = state_steps/2
magnitude = 1 / np.sqrt((2*np.pi) * (sigma**2))
state_transition = magnitude * np.exp(-0.5*((angular_dist_after_action_positive_extent/sigma)**2))
state_transition_normed = state_transition / state_transition.sum(axis=1)
state_transition_normed.round(3)

array([[0.   , 0.881, 0.119, 0.   ],
       [0.   , 0.   , 0.881, 0.119],
       [0.119, 0.   , 0.   , 0.881],
       [0.881, 0.119, 0.   , 0.   ]])

In [72]:
(alpha_t1 @ state_transition_normed)

array([0.24997538, 0.17662192, 0.16669128, 0.24004474])

In [73]:
alpha_t2_o0 = (alpha_t1 @ state_transition_normed) * B[0]
alpha_t2_o0

array([0.16665026, 0.11774795, 0.16669128, 0.24004474])

In [74]:
alpha_t2_o1 = (alpha_t1 @ state_transition_normed) * B[1]
alpha_t2_o1

array([0.08332513, 0.05887397, 0.        , 0.        ])

In [75]:
candidate_step = -np.pi/2

angular_dist_between_states_negative_extent = angular_dist_between_states % (-2*np.pi)
angular_dist_after_action = (angular_dist_between_states_negative_extent - candidate_step)
angular_dist_after_action_negative_extent = angular_dist_after_action % (-2*np.pi)
sigma = state_steps/2
magnitude = 1 / np.sqrt((2*np.pi) * (sigma**2))
state_transition = magnitude * np.exp(-0.5*((angular_dist_after_action_negative_extent/sigma)**2))
state_transition_normed = state_transition / state_transition.sum(axis=1)
state_transition_normed.round(3)

array([[0.   , 0.   , 0.119, 0.881],
       [0.881, 0.   , 0.   , 0.119],
       [0.119, 0.881, 0.   , 0.   ],
       [0.   , 0.119, 0.881, 0.   ]])

In [76]:
(alpha_t2_o1 @ np.round(state_transition_normed, 2))

array([0.0518091 , 0.        , 0.00999902, 0.08039099])

In [77]:
alpha_t3_o0 = (alpha_t2_o1 @ np.round(state_transition_normed, 2)) * B[0]
alpha_t3_o0, alpha_t3_o0.sum()

(array([0.0345394 , 0.        , 0.00999902, 0.08039099]),
 np.float64(0.12492940336066481))

In [78]:
alpha_t3_o1 = (alpha_t2_o1 @ np.round(state_transition_normed, 2)) * B[1]
alpha_t3_o1, alpha_t3_o1.sum()

(array([0.0172697, 0.       , 0.       , 0.       ]),
 np.float64(0.017269699305816802))

In [79]:
likelihood_t3_o0 = (alpha_t3_o0).sum()
likelihood_t3_o0_given_o_seq = likelihood_t3_o0 / likelihood_t2_o1
likelihood_t3_o0, likelihood_t3_o0_given_o_seq

(np.float64(0.12492940336066481), np.float64(0.8785526843560911))

In [80]:
likelihood_t3_o1 = (alpha_t3_o1).sum()
likelihood_t3_o1_given_o_seq = likelihood_t3_o1 / likelihood_t2_o1
likelihood_t3_o1, likelihood_t3_o1_given_o_seq

(np.float64(0.017269699305816802), np.float64(0.12144731564390891))

In [81]:
post_t3_o0 = (likelihood_t3_o0_given_o_seq * post_t2_o1) / ((likelihood_t3_o0_given_o_seq * post_t2_o1) + (0.5 * (1-post_t2_o1)))
post_t3_o0

np.float64(0.8332548556879074)

In [82]:
post_t3_o1 = (likelihood_t3_o1_given_o_seq * post_t2_o1) / ((likelihood_t3_o1_given_o_seq * post_t2_o1) + (0.5 * (1-post_t2_o1)))
post_t3_o1

np.float64(0.4085597860744727)

In [83]:
entropy_t3_o0 = -((post_t3_o0*np.log2(post_t3_o0)) + ((1-post_t3_o0)*np.log2(1-post_t3_o0)))
entropy_t3_o0

np.float64(0.6502046091154393)

In [84]:
entropy_t3_o1 = -((post_t3_o1*np.log2(post_t3_o1)) + ((1-post_t3_o1)*np.log2(1-post_t3_o1)))
entropy_t3_o1

np.float64(0.9757380377761071)

In [85]:
p_t3_o0 = (post_t2_o1*likelihood_t3_o0_given_o_seq) + ((1-post_t2_o1)*(1/2))
p_t3_o0

np.float64(0.7800733784945584)

In [86]:
p_t3_o1 = (post_t2_o1*likelihood_t3_o1_given_o_seq) + ((1-post_t2_o1)*(1/2))
p_t3_o1

np.float64(0.21992662150544157)

In [87]:
(p_t3_o0*entropy_t3_o0) + (p_t3_o1*entropy_t3_o1)

np.float64(0.7217980762678626)

In [99]:
candidate_step = np.pi/2

angular_dist_between_states_positive_extent = angular_dist_between_states % (2*np.pi)
angular_dist_after_action = (angular_dist_between_states_positive_extent - candidate_step)
angular_dist_after_action_positive_extent = angular_dist_after_action % (2*np.pi)
sigma = state_steps/2
magnitude = 1 / np.sqrt((2*np.pi) * (sigma**2))
state_transition = magnitude * np.exp(-0.5*((angular_dist_after_action_positive_extent/sigma)**2))
state_transition_normed = state_transition / state_transition.sum(axis=1)
state_transition_normed.round(3)

array([[0.   , 0.881, 0.119, 0.   ],
       [0.   , 0.   , 0.881, 0.119],
       [0.119, 0.   , 0.   , 0.881],
       [0.881, 0.119, 0.   , 0.   ]])

In [100]:
alpha_t3_o0 = (alpha_t2_o1 @ np.round(state_transition_normed, 2)) * B[0]
alpha_t3_o0, alpha_t3_o0.sum()

(array([0.        , 0.04888407, 0.06180811, 0.00706488]),
 np.float64(0.11775706519013047))

In [101]:
alpha_t3_o1 = (alpha_t2_o1 @ np.round(state_transition_normed, 2)) * B[1]
alpha_t3_o1, alpha_t3_o1.sum()

(array([0.        , 0.02444204, 0.        , 0.        ]),
 np.float64(0.02444203747635114))

In [102]:
likelihood_t3_o0 = (alpha_t3_o0).sum()
likelihood_t3_o0_given_o_seq = likelihood_t3_o0 / likelihood_t2_o1
likelihood_t3_o0, likelihood_t3_o0_given_o_seq

(np.float64(0.11775706519013047), np.float64(0.8281139823105755))

In [103]:
likelihood_t3_o1 = (alpha_t3_o1).sum()
likelihood_t3_o1_given_o_seq = likelihood_t3_o1 / likelihood_t2_o1
likelihood_t3_o1, likelihood_t3_o1_given_o_seq

(np.float64(0.02444203747635114), np.float64(0.17188601768942444))

In [104]:
post_t3_o0 = (likelihood_t3_o0_given_o_seq * post_t2_o1) / ((likelihood_t3_o0_given_o_seq * post_t2_o1) + (0.5 * (1-post_t2_o1)))
post_t3_o0

np.float64(0.8248773189144519)

In [105]:
post_t3_o1 = (likelihood_t3_o1_given_o_seq * post_t2_o1) / ((likelihood_t3_o1_given_o_seq * post_t2_o1) + (0.5 * (1-post_t2_o1)))
post_t3_o1

np.float64(0.4943574076622981)

In [106]:
entropy_t3_o0 = -((post_t3_o0*np.log2(post_t3_o0)) + ((1-post_t3_o0)*np.log2(1-post_t3_o0)))
entropy_t3_o0

np.float64(0.6692902022692799)

In [107]:
entropy_t3_o1 = -((post_t3_o1*np.log2(post_t3_o1)) + ((1-post_t3_o1)*np.log2(1-post_t3_o1)))
entropy_t3_o1

np.float64(0.9999081305528608)

In [108]:
p_t3_o0 = (post_t2_o1*likelihood_t3_o0_given_o_seq) + ((1-post_t2_o1)*(1/2))
p_t3_o0

np.float64(0.7427561482316247)

In [109]:
p_t3_o1 = (post_t2_o1*likelihood_t3_o1_given_o_seq) + ((1-post_t2_o1)*(1/2))
p_t3_o1

np.float64(0.2572438517683753)

In [110]:
(p_t3_o0*entropy_t3_o0) + (p_t3_o1*entropy_t3_o1)

np.float64(0.7543396316046287)

In [142]:
(88*12) + (12*17)

1260